## 선택 분류 문제의 신경망 처리
- 선택 분류 문제는 몇 가지 정해진 후보 가운데 하나를 골라 답하는 문제
- 2개를 판정하는 문제 -> 이진 판단 문제
- 퍼셉트론 하나가 후보 하나에 대한 로짓값을 출력하기 때문에, 후보 수 만큼 퍼셉트론이 필요
    - 로짓: 실제 표현하려는 값을 로그값으로 나타낸 값

## 소프트맥스 함수
- 로짓값 벡터를 확률 분포 벡터로 변환해주는 비선형 함수
- 로짓값들은 후보 항들에 대한 선택 확률을 로그 척도로 표현한 것, 따라서 다시 로그 계산을 하지 않아도 최대항만 확인하면 됨
- 손실 함숫값도 '소프트맥스 교차 엔트로피'를 이용해 바로 계산할 수 있음
- 로짓값을 확률값으로 바꿔줘야 하는 이유
    - 사용자가 시각 분포를 눈으로 확인 할 수 있음
    - 손실 함수에 대한 편미분을 구해 역전파를 진행할 때 이러한 확률 분포가 필요
- 따라서 로짓값을 확률값으로 바꿔주는 소프트맥스 함수가 필요
- 소프트맥스 함수의 입력은 로짓값 벡터
- 소프트맥스 함수의 출력은 확률값 벡터

In [5]:
%run Chapter_1.ipynb

Epoch 1: loss=33.875, accuracy=0.557/0.812
Epoch 2: loss=8.226, accuracy=0.820/0.814
Epoch 3: loss=7.582, accuracy=0.812/0.809
Epoch 4: loss=7.475, accuracy=0.808/0.811
Epoch 5: loss=7.395, accuracy=0.810/0.809
Epoch 6: loss=7.328, accuracy=0.808/0.810
Epoch 7: loss=7.269, accuracy=0.808/0.811
Epoch 8: loss=7.217, accuracy=0.808/0.812
Epoch 9: loss=7.175, accuracy=0.810/0.810
Epoch 10: loss=7.135, accuracy=0.809/0.810

Final Test: final accuracy = 0.810
[[1.02697603]
 [1.47450981]
 [1.66960135]
 [2.04468668]
 [1.62513525]
 [0.60292627]
 [2.39993815]
 [0.54107313]
 [0.46878034]
 [1.01969382]]
[4.16894769]
Epoch 20: loss=5.804, accuracy=0.825/0.831
Epoch 40: loss=5.259, accuracy=0.834/0.828
Epoch 60: loss=5.056, accuracy=0.837/0.838
Epoch 80: loss=4.950, accuracy=0.838/0.840
Epoch 100: loss=4.910, accuracy=0.840/0.826

Final Test: final accuracy = 0.826


In [6]:
def steel_exec(epoch_count=10, mb_size=10, report=1):
    load_steel_dataset()
    init_model()
    train_and_test(epoch_count, mb_size, report)

In [7]:
def load_steel_dataset():
    with open('../jch/chap03/faults.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        rows = []
        for row in csvreader:
            rows.append(row)
            
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 27, 7
    data = np.asarray(rows, dtype='float32')

In [8]:
def forward_postproc(output, y):
    entropy = softmax_cross_entropy_with_logits(y, output)
    loss = np.mean(entropy) 
    return loss, [y, output, entropy]

def backprop_postproc(G_loss, aux):
    y, output, entropy = aux
    
    g_loss_entropy = 1.0 / np.prod(entropy.shape)
    g_entropy_output = softmax_cross_entropy_with_logits_derv(y, output)
    
    G_entropy = g_loss_entropy * G_loss
    G_output = g_entropy_output * G_entropy
    
    return G_output

In [9]:
def eval_accuracy(output, y):
    estimate = np.argmax(output, axis=1)
    answer = np.argmax(y, axis=1)
    correct = np.equal(estimate, answer)
    
    return np.mean(correct)

In [10]:
def softmax(x):
    max_elem = np.max(x, axis=1)
    diff = (x.transpose() - max_elem).transpose()
    exp = np.exp(diff)
    sum_exp = np.sum(exp, axis=1)
    probs = (exp.transpose() / sum_exp).transpose()
    return probs

def softmax_derv(x, y):
    mb_size, nom_size = x.shape
    derv = np.ndarray([mb_size, nom_size, nom_size])
    for n in range(mb_size):
        for i in range(nom_size):
            for j in range(nom_size):
                derv[n, i, j] = -y[n,i] * y[n,j]
            derv[n, i, i] += y[n,i]
    return derv

def softmax_cross_entropy_with_logits(labels, logits):
    probs = softmax(logits)
    return -np.sum(labels * np.log(probs+1.0e-10), axis=1)

def softmax_cross_entropy_with_logits_derv(labels, logits):
    return softmax(logits) - labels

In [14]:
steel_exec()

Epoch 1: loss=16.593, accuracy=0.279/0.271
Epoch 2: loss=15.747, accuracy=0.316/0.425
Epoch 3: loss=14.603, accuracy=0.366/0.307
Epoch 4: loss=15.316, accuracy=0.335/0.315
Epoch 5: loss=15.509, accuracy=0.326/0.379
Epoch 6: loss=15.479, accuracy=0.328/0.304
Epoch 7: loss=15.242, accuracy=0.338/0.315
Epoch 8: loss=15.539, accuracy=0.325/0.427
Epoch 9: loss=15.301, accuracy=0.335/0.164
Epoch 10: loss=15.880, accuracy=0.310/0.205

Final Test: final accuracy = 0.205


In [12]:
LEARNING_RATE = 0.0001
steel_exec()

Epoch 1: loss=16.564, accuracy=0.281/0.248
Epoch 2: loss=15.821, accuracy=0.313/0.358
Epoch 3: loss=15.687, accuracy=0.319/0.355
Epoch 4: loss=15.583, accuracy=0.323/0.419
Epoch 5: loss=15.093, accuracy=0.345/0.207
Epoch 6: loss=15.152, accuracy=0.342/0.358
Epoch 7: loss=15.628, accuracy=0.321/0.419
Epoch 8: loss=15.539, accuracy=0.325/0.189
Epoch 9: loss=15.539, accuracy=0.325/0.353
Epoch 10: loss=15.390, accuracy=0.332/0.435

Final Test: final accuracy = 0.435
